In [1]:
import numpy as np
import pandas as pd
from linearmodels import PanelOLS
from linearmodels import RandomEffects

#data is unique by gvkey and fyear
data = pd.read_csv('fundamentals_annual/fundamentals_annual.csv')

income = pd.read_csv('fundamentals_annual/income.csv')
data = data.merge(income, on = ['gvkey','fyear'],  suffixes=('', '_drop'))
data = data[[c for c in data.columns if not c.endswith('_drop')]]

shares = pd.read_csv('shares.csv')
data = data.merge(shares, on = ['gvkey','fyear'],  suffixes=('', '_drop'))
data = data[[c for c in data.columns if not c.endswith('_drop')]]

data.drop(['consol','popsrc','indfmt'],axis=1,inplace=True) #same for all rows
data.drop(['dvpd','opiti','tii','uopi'], axis=1, inplace=True) #NaN values only
data.drop(['datadate','tic','conm','fyr'],axis=1,inplace=True) #dont need these fields

data.sort_values(by=['gvkey','fyear'],inplace=True) #sort by gvkey and fyear

In [2]:
market_data = pd.read_csv('market_data.csv')
# data = data.merge(market_data, left_on= 'fyear', right_on='caldt')

In [3]:
#define a threshold for missing values
perc = 5.0 
min_count =  int(((100-perc)/100)*data.shape[0] + 1)
data = data.dropna( axis=1, thresh=min_count)

#drop columns with zero variance
for i in data.columns:
    if(len(data[i].unique()) == 1):
        data.drop(i, axis=1,inplace=True)

#row-wise NA dropping
data.dropna(inplace=True)

In [4]:
#shifting the NI column to remove look ahead bias
df = data[data.columns]
ni = df.groupby('gvkey')['ni'].shift(-1)
df['ni'] = ni
df.dropna(inplace=True)

In [5]:
df

,gvkey,fyear,ap,at,ch,cshpri,dltt,dvt,ebit,ebitda,...,opeps,revt,seq,txdi,txp,txt,sic,ni,pi,csho
0,2080,2000,20.310,346.680,3.259,11.813,45.000,9.497,7.234,17.150,...,1.49,367.444,249.140,1.708,0.000,4.671,2511,-2.642,15.067,11.765
1,2080,2001,15.010,301.403,5.347,11.702,7.482,9.378,-2.206,8.396,...,0.01,305.676,234.472,-0.824,0.000,-1.042,2511,6.741,-3.684,11.727
2,2080,2002,17.738,290.880,1.371,11.698,3.000,9.358,7.507,18.311,...,0.65,323.487,229.643,2.215,0.000,2.369,2511,-0.470,9.110,11.661
3,2080,2003,15.127,280.380,15.181,11.609,0.000,9.261,1.970,12.119,...,0.56,316.857,220.018,-1.154,1.530,0.462,2511,8.209,4.867,11.600
4,2080,2004,19.948,297.366,4.022,11.687,15.604,9.355,3.897,13.928,...,0.72,315.654,221.212,1.875,0.682,2.641,2511,55.342,10.850,11.736
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2755,271841,2015,105.765,1707.456,20.872,183.786,275.000,4.466,29.368,41.345,...,0.10,627.710,781.828,-4.399,25.777,1.814,1520,-357.677,19.741,183.741
2756,271841,2016,87.455,1601.527,9.078,181.488,200.920,0.000,-129.176,-118.758,...,-1.03,302.368,592.747,27.468,15.936,30.764,1520,-256.591,-154.291,181.953
2757,271841,2017,77.026,868.977,8.613,13.446,161.725,0.000,-159.837,-150.375,...,-19.14,198.579,228.120,-7.829,14.018,-6.974,1520,-108.368,-293.292,13.551
2758,271841,2018,60.239,652.563,8.344,20.574,139.751,0.000,-32.363,-26.864,...,-3.31,270.746,126.912,-6.483,14.795,-5.618,1520,-6.478,-114.438,19.892


In [6]:
""" Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
"""
def stepwise_selection(df, X, y, reg_model, initial_list=[], threshold_in=0.01, threshold_out = 0.05, verbose=True):
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            exog = sm.add_constant(pd.DataFrame(X[included+[new_column]]))
            model = reg_model(df.ni, exog).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        model = reg_model(df.ni, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included



In [7]:
#Panel Regression data preparation
import statsmodels.api as sm
from linearmodels.panel import PooledOLS
cols = list(df.columns)
cols.remove('gvkey')
cols.remove('fyear')
cols.remove('ni')
gvkey = data['gvkey']
fyear = data['fyear']
df['fyear'] = fyear
df['gvkey'] = gvkey
df = df.set_index(['gvkey','fyear'])

In [8]:
X = df.drop('ni',axis=1)
y=df['ni']
result = stepwise_selection(df,X,y,reg_model = PooledOLS)
exog_vars = result
exog = sm.add_constant(df[exog_vars])
mod = PooledOLS(df.ni, exog)
pooled_res = mod.fit()
print(pooled_res)
print()
print('Rsquared')
print(pooled_res.rsquared)

/var/folders/y3/v6fyb_rs72bcn7ftc3lvwvmw0000gn/T/ipykernel_20659/1392687859.py:19: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=excluded)


Add  lt                             with p-value 0.0
Add  csho                           with p-value 0.0
Add  at                             with p-value 0.0
Add  gp                             with p-value 0.0
Drop csho                           with p-value 0.951395
Add  ch                             with p-value 0.0
Drop lt                             with p-value 0.300879
Add  dltt                           with p-value 0.0
Add  revt                           with p-value 0.0
Add  dvt                            with p-value 0.0
Add  ebitda                         with p-value 0.0
Drop at                             with p-value 0.0613924
Add  invt                           with p-value 0.0
Add  ap                             with p-value 0.0
Add  ebit                           with p-value 0.0
Add  txt                            with p-value 3.75496e-06
Add  cshpri                         with p-value 5.23615e-05
Add  pi                             with p-value 0.00485021
       

In [9]:
#RandomEffects Regression
from linearmodels.panel import RandomEffects
result = stepwise_selection(df,X,y,reg_model = RandomEffects)
exog_vars = result
exog = sm.add_constant(df[exog_vars])
mod = RandomEffects(df.ni, exog)
re_res = mod.fit()
print(re_res)
print()
print('Rsquared')
print(re_res.rsquared)

/var/folders/y3/v6fyb_rs72bcn7ftc3lvwvmw0000gn/T/ipykernel_20659/1392687859.py:19: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=excluded)


Add  lt                             with p-value 0.0
Add  at                             with p-value 0.0
Add  gp                             with p-value 0.0
Add  ch                             with p-value 0.0
Drop lt                             with p-value 0.300879
Add  dltt                           with p-value 0.0
Add  revt                           with p-value 0.0
Add  dvt                            with p-value 0.0
Add  ebitda                         with p-value 0.0
Drop at                             with p-value 0.0613924
Add  invt                           with p-value 0.0
Add  ap                             with p-value 0.0
Add  ebit                           with p-value 0.0
Add  txt                            with p-value 3.75496e-06
Add  cshpri                         with p-value 5.23615e-05
Add  pi                             with p-value 0.00485021
                        RandomEffects Estimation Summary                        
Dep. Variable:                     ni

In [10]:
#Between OLS regression
from linearmodels.panel import BetweenOLS
result = stepwise_selection(df,X,y,reg_model = BetweenOLS)
exog_vars = result
exog = sm.add_constant(df[exog_vars])
mod = BetweenOLS(df.ni, exog)
be_res = mod.fit()
print(be_res)
print()
print('Rsquared')
print(be_res.rsquared)

/var/folders/y3/v6fyb_rs72bcn7ftc3lvwvmw0000gn/T/ipykernel_20659/1392687859.py:19: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=excluded)


Add  gp                             with p-value 0.0
Add  pi                             with p-value 0.0
Add  lt                             with p-value 0.0
Add  revt                           with p-value 0.0
Add  txt                            with p-value 6.24026e-10
Add  ch                             with p-value 1.19431e-05
Add  opeps                          with p-value 6.23469e-05
                         BetweenOLS Estimation Summary                          
Dep. Variable:                     ni   R-squared:                        0.9958
Estimator:                 BetweenOLS   R-squared (Between):              0.9958
No. Observations:                 185   R-squared (Within):               0.0909
Date:                Mon, Feb 07 2022   R-squared (Overall):              0.4170
Time:                        13:13:58   Log-likelihood                   -842.63
Cov. Estimator:            Unadjusted                                           
                                      

In [11]:
def stepwise_selection_panel(df, X, y, reg_model, drop_absorbed =True, entity_effects =False, time_effects = False, initial_list=[], threshold_in=0.01, threshold_out = 0.07, verbose=True):
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            exog = sm.add_constant(pd.DataFrame(X[included+[new_column]]))
            model = reg_model(df.ni, exog, entity_effects = entity_effects, time_effects = time_effects, drop_absorbed=True).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        model = reg_model(df.ni, sm.add_constant(pd.DataFrame(X[included])), drop_absorbed =True, entity_effects =False, time_effects = False).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

In [12]:
#PanelOLS regression - entity effects is true
# result = stepwise_selection_panel(df.drop(['sic','invt'],axis=1),X.drop(['sic','invt'],axis=1),y,reg_model = PanelOLS, entity_effects = True)
exog_vars = result #df.drop('ni',axis=1).columns
exog = sm.add_constant(df[exog_vars])
mod = PanelOLS(df.ni, exog, entity_effects=True, drop_absorbed=True)
mod = mod.fit()
print(mod)

                          PanelOLS Estimation Summary                           
Dep. Variable:                     ni   R-squared:                        0.4373
Estimator:                   PanelOLS   R-squared (Between):             -3.5940
No. Observations:                2333   R-squared (Within):               0.4373
Date:                Mon, Feb 07 2022   R-squared (Overall):             -1.0397
Time:                        13:13:58   Log-likelihood                -1.752e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      237.68
Entities:                         185   P-value                           0.0000
Avg Obs:                       12.611   Distribution:                  F(7,2141)
Min Obs:                       1.0000                                           
Max Obs:                       21.000   F-statistic (robust):             237.68
                            

In [13]:
# #time effects is true
# result = stepwise_selection_panel(df.drop(['sic','invt','cpiret','SNP500','SNP500_CD','caldt','t90ret','b5ret'],axis=1),
#                                     X.drop(['sic','invt','cpiret','SNP500','SNP500_CD','caldt','t90ret','b5ret'],axis=1),y,reg_model = PanelOLS, entity_effects = True, time_effects=True)
# exog_vars = result #df.drop('ni',axis=1).columns
# exog = sm.add_constant(df[exog_vars])
# mod = PanelOLS(df.ni, exog, entity_effects=True, time_effects=True, drop_absorbed =True)
# fe_te_res = mod.fit()
# print(fe_te_res)
# print()
# print('Rsquared')
# print(fe_te_res.rsquared)